<a href="https://colab.research.google.com/github/Armin-Abdollahi/Signal-Processing/blob/main/Converts_each_window_of_the_EEG_signal_into_spectrogram_images.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Using the MNE library to normalize the EEG data and segment it into smaller windows

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install --upgrade mne

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 45.3 MB/s eta 0:00:00


In [3]:
import mne
import numpy as np

In [4]:
# Load the data
file_path = '/content/drive/MyDrive/BCI_Database/gdf/A1_CE_baseline.gdf'
raw = mne.io.read_raw_gdf(file_path, preload=True)

Extracting EDF parameters from /content/drive/MyDrive/BCI_Database/gdf/A1_CE_baseline.gdf...
GDF file detected
Setting channel info structure...
Could not determine channel type of the following channels, they will be set as EEG:
Fz, FCz, Cz, CPz, Pz, C1, C3, C5, C2, C4, C6, EOG1, EOG2, EOG3, EMGg, EMGd, F4, FC2, FC4, FC6, CP2, CP4, CP6, P4, F3, FC1, FC3, FC5, CP1, CP3, CP5, P3
Creating raw.info structure...
Reading 0 ... 95775  =      0.000 ...   187.061 secs...


In [5]:
# Normalize the data
data = raw.get_data()
data_mean = np.mean(data, axis=1, keepdims=True)
data_std = np.std(data, axis=1, keepdims=True)
normalized_data = (data - data_mean) / data_std

In [6]:
# Update the raw object with normalized data
raw._data = normalized_data

In [7]:
# Segment the data into smaller windows (e.g., 2-second windows)
window_size = 2  # in seconds
sfreq = raw.info['sfreq']  # sampling frequency
n_samples = int(window_size * sfreq)

In [8]:
# Create epochs
events = mne.make_fixed_length_events(raw, duration=window_size)
epochs = mne.Epochs(raw, events, tmin=0, tmax=window_size, baseline=None, preload=True)

Not setting metadata
93 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 93 events and 1025 original time points ...
0 bad epochs dropped


In [9]:
# Save the epochs if needed
epochs.save('/content/drive/MyDrive/BCI_Database/gdf/A1_CE_baseline-epo.fif', overwrite=True)

print("Data normalized and segmented into epochs.")

Data normalized and segmented into epochs.


## Converts each window of the EEG signal into spectrogram images

converts each window of the EEG signal into spectrogram images using the matplotlib and scipy libraries.

In [11]:
import mne
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import spectrogram

# Load the segmented epochs
epochs = mne.read_epochs('/content/drive/MyDrive/BCI_Database/gdf/A1_CE_baseline-epo.fif')

# Function to convert a signal to a spectrogram image
def signal_to_spectrogram(signal, sfreq, output_path):
    f, t, Sxx = spectrogram(signal, fs=sfreq, nperseg=256, noverlap=128)
    plt.pcolormesh(t, f, 10 * np.log10(Sxx), shading='gouraud')
    plt.ylabel('Frequency [Hz]')
    plt.xlabel('Time [sec]')
    plt.axis('off')  # Hide axes
    plt.savefig(output_path, bbox_inches='tight', pad_inches=0)
    plt.close()

# Iterate over each epoch and save the spectrogram image
for i, epoch in enumerate(epochs):
    for ch_idx, ch_data in enumerate(epoch):
        output_path = f'/content/drive/MyDrive/BCI_Database/gdf/spectrograms/epoch_{i}_ch_{ch_idx}.png'
        signal_to_spectrogram(ch_data, epochs.info['sfreq'], output_path)

print("Spectrogram images saved for all windows.")

Reading /content/drive/MyDrive/BCI_Database/gdf/A1_CE_baseline-epo.fif ...
Isotrak not found
    Found the data of interest:
        t =       0.00 ...    2000.00 ms
        0 CTF compensation matrices available
Not setting metadata
93 matching events found
No baseline correction applied
0 projection items activated
Spectrogram images saved for all windows.
